This notebook contains code for rendering .png images according to a predefined fsleyes orthographic view, of T2 highres based MNI normalized tumor regions overlayed on a MNI brain regions template.

In [1]:
%run utils.py
%run visualiztion.py
%run search.py

1. Specify epi_corrections output directory

In [2]:
output_directory_suffix = "2019_07_02"
# On local file system:
corrections_base_directory = "../../epi_corrections_out_" + output_directory_suffix
# On samba share (remote file sytem):
#corrections_base_directory = "/run/user/1000/gvfs/smb-share:server=192.168.1.207,share=hdd3tb1/data/IVS_EPI_BASELINE/epi_corrections_out_" + output_directory_suffix
corrections_base_directory

'../../epi_corrections_out_2019_07_02'

2. Specify output render directory (full path)

In [3]:
render_dir_full = [str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in [corrections_base_directory]][0] + "/render_rois"
render_dir_full

'/run/user/1000/gvfs/smb-share:server=192.168.1.207,share=hdd3tb1/data/IVS_EPI_BASELINE/epi_corrections_out_2019_07_02/render_rois'

3. Find the MNI-normalized, ONCOHabitats tumor ROIs

In [4]:
ONCOHabitats_results_folder = "ONCOHabitats_results"
segment_files_relative = find_segment_files(corrections_base_directory + "/" + ONCOHabitats_results_folder)
segment_paths_relative = [Path(file) for file in segment_files_relative]
segments_files_full = [str(Path.joinpath(Path.cwd().parent.parent, *relative.parts[2:])) for relative in segment_paths_relative]

4. Find the corresponding MNI label files 
(should be identical to each other, but all found for convenience..)

In [7]:
[raw_label_files_e1, \
 topup_label_files_e1, \
 epic_label_files_e1, \
 raw_label_files_e2, \
 topup_label_files_e2, \
 epic_label_files_e2] = find_label_files(corrections_base_directory)

In [8]:
[raw_label_files_e1_full, \
 topup_label_files_e1_full, \
 epic_label_files_e1_full, \
 raw_label_files_e2_full, \
 topup_label_files_e2_full, \
 epic_label_files_e2_full] = \
[[str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in raw_label_files_e1], \
 [str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in topup_label_files_e1], \
 [str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in epic_label_files_e1], \
 [str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in raw_label_files_e2], \
 [str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in topup_label_files_e2], \
 [str(Path.joinpath(Path.cwd().parent.parent, *Path(relative).parts[2:])) for relative in epic_label_files_e2]]

In [9]:
# A check
print("Equal number of detected label files for raw (uncorrected), topup, and epic correction methods: %r" % \
      (len(raw_label_files_e1_full) == \
       len(raw_label_files_e2_full) == \
       len(topup_label_files_e1_full) == \
       len(topup_label_files_e2_full) == \
       len(epic_label_files_e1_full) == \
       len(epic_label_files_e2_full)))
print("Number of subject label files: %i" % len(raw_label_files_e1_full))

Equal number of detected label files for raw (uncorrected), topup, and epic correction methods: True
Number of subject label files: 45


5. For the raw, topup and epic Gradient Echo (GE) and Spin Echo (SE) nrCBV data + MNI regions file, render .png images in a predefined orthographic view, then make a video. __Note:__ The following cell must run within a fsl X environment

In [10]:
%%bash -s "{" ".join(segments_files_full)}" "{" ".join(topup_label_files_e2_full)}" "{" ".join([render_dir_full])}"
IFS=' ' read -r -a segments_files_full <<< "$1"
IFS=' ' read -r -a topup_label_files_e2_full <<< "$2"
IFS=' ' read -r -a render_dir_full <<< "$3"
render() {
    local -n _segments_files=$1
    local -n _label_files=$2
    local -n _render_dir=$3
    
    # Remove existing render directory.
    if [ -d "$_render_dir" ]; then rm -rd "$_render_dir"; fi
    
    mkdir -p "$_render_dir"
    
    for index in "${!_segments_files[@]}"
    do
        segments_file="${_segments_files[index]}"
        label_file="${_label_files[index]}"
        
        segments_file_name="$(basename $segments_file)"
        label_file_name="$(basename $label_file)"
        
        render_command='fsleyes 
            render 
            --outfile "'$_render_dir'/'output_$(printf "%03d\n" $index)'" 
            --scene ortho 
            --worldLoc 9.918212890625e-05 -18.000099182128906 7.999900817871094 
            --displaySpace 
            '$segments_file' 
            --xcentre -0.00000 -0.11019 
            --ycentre -0.00000 -0.11019 
            --zcentre -0.00000 -0.00000 
            --xzoom 803.0573770888577 
            --yzoom 809.4712464952298 
            --zzoom 803.057377088858 
            --hideLabels 
            --labelSize 14 
            --layout horizontal 
            --hideCursor 
            --bgColour 0.0 0.0 0.0 
            --fgColour 1.0 1.0 1.0 
            --cursorColour 0.0 1.0 0.0 
            --colourBarLocation top 
            --colourBarLabelSide top-left 
            --performance 3 
            '$segments_file' 
            --name '$segments_file_name' 
            --overlayType volume 
            --alpha 100.0 
            --brightness 64.94720130619707 
            --contrast 79.89440261239416 
            --cmap cool 
            --negativeCmap greyscale 
            --displayRange 0.0 12.0 
            --clippingRange 0.0 30.140860195159913 
            --gamma 0.0 
            --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0 
            '$label_file' 
            --name '$label_file_name' 
            --overlayType volume 
            --alpha 37.33333333798995 
            --brightness 50.0 
            --contrast 50.0 
            --cmap greyscale 
            --negativeCmap greyscale 
            --displayRange 0.0 207.0 
            --clippingRange 0.0 209.07 
            --gamma 0.0 
            --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0'
        echo $render_command >> "$_render_dir/commands.txt"
        #echo $render_command
        eval $render_command
        echo "--Finished rendering volume $index to .png--"
    done
    ffmpeg -framerate 1 -i "$_render_dir"/output_%03d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -preset slow -profile:v high -level:v 4.0 -pix_fmt yuv420p -crf 1 "$_render_dir"/video.mp4
    echo "--Finished rendering pngs to video--"
}
render segments_files_full topup_label_files_e2_full render_dir_full

--Finished rendering volume 0 to .png--
--Finished rendering volume 1 to .png--
--Finished rendering volume 2 to .png--
--Finished rendering volume 3 to .png--
--Finished rendering volume 4 to .png--
--Finished rendering volume 5 to .png--
--Finished rendering volume 6 to .png--
--Finished rendering volume 7 to .png--
--Finished rendering volume 8 to .png--
--Finished rendering volume 9 to .png--
--Finished rendering volume 10 to .png--
--Finished rendering volume 11 to .png--
--Finished rendering volume 12 to .png--
--Finished rendering volume 13 to .png--
--Finished rendering volume 14 to .png--
--Finished rendering volume 15 to .png--
--Finished rendering volume 16 to .png--
--Finished rendering volume 17 to .png--
--Finished rendering volume 18 to .png--
--Finished rendering volume 19 to .png--
--Finished rendering volume 20 to .png--
--Finished rendering volume 21 to .png--
--Finished rendering volume 22 to .png--
--Finished rendering volume 23 to .png--
--Finished rendering volum


(r_e2_labels_Neuromorphometrics.nii:8021): Gtk-CRITICAL **: 14:53:51.619: gtk_window_resize: assertion 'width > 0' failed
14:53:51: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(r_e2_labels_Neuromorphometrics.nii:8043): Gtk-CRITICAL **: 14:53:57.763: gtk_window_resize: assertion 'width > 0' failed
14:53:57: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(r_e2_labels_Neuromorphometrics.nii:8065): Gtk-CRITICAL **: 14:54:03.613: gtk_window_resize: assertion 'width > 0' failed
14:54:03: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(r_e2_labels_Neuromorphometrics.nii: